### 將上傳的資料解壓縮

In [1]:
!unzip homework_data.zip

Archive:  homework_data.zip
replace homework_data/speak00_data_enroll.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace homework_data/speak00_data_eval.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


### 載入資料

In [1]:
import numpy as np

enroll = []
eval = []
for i in range(10):
    enroll.append(np.load("./homework_data/speak0{}_data_enroll.npy".format(i)))
    eval.append(np.load("./homework_data/speak0{}_data_eval.npy".format(i)))

### 訂定相似度函數

In [2]:
def cosine(As,Bs):
    As_scale = np.reshape(np.linalg.norm(As,axis=1),(-1,1))
    Bs_scale = np.reshape(np.linalg.norm(Bs,axis=1),(1,-1))
    scale = np.dot(As_scale,Bs_scale)
    AB = np.matmul(As,np.transpose(Bs,(1,0)))
    return AB/scale

### 計算相似度並建立混淆矩陣

In [3]:
confusion_matrix = np.zeros((len(enroll),len(enroll)),dtype=int)
for eval_idx in range(len(eval)):
    scores = []
    for enroll_idx in range(len(enroll)):
        # scores.append(np.max(cosine(enroll[enroll_idx],eval[eval_idx]),axis=0))
        # scores.append(np.min(cosine(enroll[enroll_idx],eval[eval_idx]),axis=0))
        scores.append(np.mean(cosine(enroll[enroll_idx],eval[eval_idx]),axis=0))
    predict_classes = np.argmax(scores,axis=0)
    for predict_class_idx in predict_classes:
       confusion_matrix[predict_class_idx][eval_idx] += 1 
    # print(classes)

confusion_matrix

array([[ 55,   0,   0,   0,   3,   1,   0,   0,   2,   0],
       [  0, 109,   2,   0,   0,   0,   0,   0,   0,   0],
       [  0,  12, 130,   5,   2,   0,   0,   0,  14,   0],
       [  7,   0,   7,  90,   2,   0,   0,   0,   0,   0],
       [  2,   0,   0,   0,  30,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 115,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,  71,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  96,   0,   0],
       [  0,   0,   0,   0,   2,   0,   0,   0,  27,   0],
       [  3,   0,  22,   3,  10,   0,   0,   0,   6,  42]])

In [16]:
precision = np.sum(confusion_matrix*np.eye(len(enroll)),axis=0)/np.reshape(np.sum(confusion_matrix,axis=1),(-1))

In [17]:
recall = np.sum(confusion_matrix*np.eye(len(enroll)),axis=0)/np.sum(confusion_matrix,axis=0)

### 建立 csv table

In [29]:
table = [[""]]
for label_idx in range(len(enroll)):
    table[0].append("actual:{}".format(label_idx))

for predict_idx in range(len(enroll)):
    table.append(["predict:{}".format(predict_idx)])
    for label_idx in range(len(enroll)):
        table[predict_idx+1].append(confusion_matrix[predict_idx][label_idx])

table[0].append("precision")
for idx in range(len(enroll)):
    table[idx+1].append(precision[idx])

table.append(["recall"])
print(table)
for idx in range(len(enroll)):
    table[len(enroll)+1].append(recall[idx])
table[len(enroll)+1].append('')


[['', 'actual:0', 'actual:1', 'actual:2', 'actual:3', 'actual:4', 'actual:5', 'actual:6', 'actual:7', 'actual:8', 'actual:9', 'precision'], ['predict:0', 55, 0, 0, 0, 3, 1, 0, 0, 2, 0, 0.9016393442622951], ['predict:1', 0, 109, 2, 0, 0, 0, 0, 0, 0, 0, 0.9819819819819819], ['predict:2', 0, 12, 130, 5, 2, 0, 0, 0, 14, 0, 0.7975460122699386], ['predict:3', 7, 0, 7, 90, 2, 0, 0, 0, 0, 0, 0.8490566037735849], ['predict:4', 2, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0.9375], ['predict:5', 0, 0, 0, 0, 0, 115, 0, 0, 0, 0, 1.0], ['predict:6', 0, 0, 0, 0, 0, 0, 71, 0, 0, 0, 1.0], ['predict:7', 0, 0, 0, 0, 0, 0, 0, 96, 0, 0, 1.0], ['predict:8', 0, 0, 0, 0, 2, 0, 0, 0, 27, 0, 0.9310344827586207], ['predict:9', 3, 0, 22, 3, 10, 0, 0, 0, 6, 42, 0.4883720930232558], ['recall']]


### 寫入 csv 檔

In [27]:
import csv
with open('confusion_matrix.csv', 'w', newline='') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerows(table)